# Financial News Summarizer + Sentiment Annotator using DeepSeek-R1 via Ollama local (RSS + Full Article)

In [1]:
import os
import feedparser
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from datetime import datetime
import pandas as pd
import time
import openai
import streamlit as st
import matplotlib.pyplot as plt

In [2]:
# STEP 1: Set up Ollama client for DeepSeek
ollama_via_openai = openai.OpenAI(
    base_url='http://localhost:11434/v1',
    api_key='ollama'
)
MODEL = "deepseek-r1:1.5b"

In [3]:
# STEP 2: Parse RSS feed with full article links
RSS_FEED_URL = "https://feeds.finance.yahoo.com/rss/2.0/headline?s=^GSPC&region=US&lang=en-US"

def fetch_rss_entries():
    feed = feedparser.parse(RSS_FEED_URL)
    entries = [(entry.title, entry.link) for entry in feed.entries]
    return entries

In [4]:
# STEP 3: Scrape full article content from URL (basic paragraph grabber)
def fetch_article_content(url):
    try:
        req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
        with urlopen(req) as response:
            soup = BeautifulSoup(response.read(), 'html.parser')
        paragraphs = soup.find_all('p')
        text = " ".join(p.get_text() for p in paragraphs)
        return text.strip()
    except Exception as e:
        return f"Error fetching article: {e}"

In [5]:
# STEP 4: Use DeepSeek via Ollama to summarize + analyze sentiment
def analyze_article_with_ollama(headline, article):
    prompt = (
        f"""
        Headline: {headline}

        Article:
        {article}

        Summarize the article in 3 or 4 lines. 
        The summary must include what the article is about, what its conclusion is and how it arrived at the conclusion.
        Then assess the sentiment as bullish, bearish, or neutral.

        Respond in this exact format:
        Summary: <summary here>
        Sentiment: <bullish/bearish/neutral>
        """
    )
    try:
        response = ollama_via_openai.chat.completions.create(
            model=MODEL,
            messages=[{"role": "user", "content": prompt}]
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error: {e}"

In [6]:
# STEP 5: Parse response
def parse_response(response_text):
    print("--- Raw DeepSeek Response ---")
    print(response_text)
    print("-----------------------------")

    summary = ""
    sentiment = ""
    for line in response_text.splitlines():
        if 'summary' in line.lower():
            summary = line.split(":", 1)[-1].strip()
        elif 'sentiment' in line.lower():
            sentiment = line.split(":", 1)[-1].strip().lower()
    return summary, sentiment

In [7]:
# STEP 6: Process full pipeline
def run_pipeline():
    entries = fetch_rss_entries()
    results = []

    for headline, link in entries:
        article = fetch_article_content(link)
        gpt_response = analyze_article_with_ollama(headline, article)
        summary, sentiment = parse_response(gpt_response)
        results.append({
            "date": datetime.now().strftime('%Y-%m-%d'),
            "headline": headline,
            "summary": summary,
            "sentiment": sentiment,
            "url": link
        })
        time.sleep(1.5)

    df = pd.DataFrame(results)
    df.to_csv("financial_news_summary_deepseek.csv", index=False)
    print(df.head())

In [8]:
if __name__ == "__main__":
    run_pipeline()

--- Raw DeepSeek Response ---
<think>
Okay, so I need to summarize this article about the P/E ratio. Let me read through it carefully. The headline explains what a P/E ratio is and how investors use it to assess investments, specifically focusing on the tech sector after recent sell-offs.

The article starts with some information about why stock markets can be volatile. StockBrokers.com director Jessica Inskip joins Wealth to explain the P/E ratio. She talks about how P/E ratios are calculated and provides examples of companies in the tech sector that have high valuations compared to their fundamentals.

In the conclusion, she suggests revisiting tech after the market drop because earnings in tech might recover faster than in other sectors. So, she's pointing out that high P/E could indicate undervaluation due to rapid growth and strong earnings. She concludes with a bullish sentiment based on this reassurance from Jessica.

Wait, I should make sure my summary is concise. The article d